# Buyer Frequencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
bid=pd.read_excel('bid19to21.xlsx')
stock=pd.read_excel('stock19to21.xlsx')

In [3]:
bid['buyer_username']=bid['buyer_username'].str.lower()
stock['buyer_username']=stock['buyer_username'].str.lower()

In [4]:
bid['win']='N'
bid.loc[bid['bid']>=bid['winning_bid'], 'win']='Y'

In [5]:
bid['bid_placed']=pd.to_datetime(bid['bid_placed'])
stock['submitted_date']=pd.to_datetime(stock['submitted_date'])

In [6]:
# bid.to_excel('bid19to21.xlsx')
# stock.to_excel('stock19to21.xlsx')

In [7]:
won=bid[(bid['win']=='Y')&(bid['bid_placed']>datetime.datetime(2020,7,14))]
won['month_yr']=won['bid_placed'].dt.strftime('%Y%m')
won['month']=won['bid_placed'].dt.month

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
date_won=won.groupby([won['buyer_username'], won['bid_placed'].dt.date, won['month_yr'], won['month']])['bid'].count().reset_index()

In [9]:
date_won=date_won.rename(columns={'bid_placed':'purchase_date'})

In [10]:
quotes=stock[(stock['submitted_date']>datetime.datetime(2020,7,14))]
quotes['month_yr']=quotes['submitted_date'].dt.strftime('%Y%m')
quotes['month']=quotes['submitted_date'].dt.month

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
date_quotes=quotes.groupby([quotes['buyer_username'], quotes['submitted_date'].dt.date, quotes['month_yr'], quotes['month']])['price'].count().reset_index()

In [12]:
date_quotes=date_quotes.rename(columns={'submitted_date':'purchase_date', 'price':'quote'})

In [13]:
all_purchase=date_won.merge(date_quotes, how='outer')
all_purchase=all_purchase.fillna(0).sort_values(by=['buyer_username','purchase_date']).reset_index(drop=True)

all_purchase['total']=all_purchase['bid']+all_purchase['quote']
all_purchase

,buyer_username,purchase_date,month_yr,month,bid,quote,total
0,adore the couture,2020-07-24,202007,7,1.0,0.0,1.0
1,adore the couture,2020-07-31,202007,7,1.0,0.0,1.0
2,adore the couture,2020-08-07,202008,8,5.0,0.0,5.0
3,adore the couture,2020-08-08,202008,8,0.0,1.0,1.0
4,adore the couture,2020-08-15,202008,8,0.0,9.0,9.0
...,...,...,...,...,...,...,...
2934,yolo luxury consignment,2021-05-03,202105,5,0.0,2.0,2.0
2935,yolo luxury consignment,2021-06-05,202106,6,1.0,0.0,1.0
2936,yolo luxury consignment,2021-06-13,202106,6,1.0,0.0,1.0
2937,zippychicks,2020-09-05,202009,9,0.0,2.0,2.0


In [14]:
# all_purchase.to_excel('all_purchases.xlsx')

In [15]:
end_date = all_purchase['purchase_date'][1]
start_date = all_purchase['purchase_date'][0]

num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

print(num_months)

0


In [16]:
locs=all_purchase.loc[all_purchase['buyer_username']=='Adore The Couture'].index

In [17]:
all_purchase['time']=0
all_purchase['time'].iloc[locs]=1
all_purchase.head()

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,buyer_username,purchase_date,month_yr,month,bid,quote,total,time
0,adore the couture,2020-07-24,202007,7,1.0,0.0,1.0,0
1,adore the couture,2020-07-31,202007,7,1.0,0.0,1.0,0
2,adore the couture,2020-08-07,202008,8,5.0,0.0,5.0,0
3,adore the couture,2020-08-08,202008,8,0.0,1.0,1.0,0
4,adore the couture,2020-08-15,202008,8,0.0,9.0,9.0,0


In [18]:
def frequency(df):
    """
    Returns df which sorts buyers by their purchase frequency
    
    Parameter df: the DataFrame of purchases
    Precondition: df is a DataFrame
    """
    df['time_btwn']='NaN'
    for b in list(df['buyer_username'].unique()):
        locs=df.loc[df['buyer_username']==b].index
        num_months=[0]
        for i in locs[1:]:
            end_date = df['purchase_date'][i]
            start_date = df['purchase_date'][i-1]
            n=(end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
            num_months.append(n)
        df['time_btwn'].iloc[locs]=num_months
        
    return df

new=frequency(all_purchase)

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
new[new['buyer_username']=='rue gilt'].groupby(['buyer_username', 'month_yr'])['total'].sum()

buyer_username  month_yr
rue gilt        202007        66.0
                202008       249.0
                202009       211.0
                202010       237.0
                202011       281.0
                202012       291.0
                202101       695.0
                202102       707.0
                202103       849.0
                202104      1110.0
                202105       692.0
                202106        41.0
                202107       159.0
Name: total, dtype: float64

In [20]:
time_btwn=new.groupby(['buyer_username','month_yr'])['time_btwn'].sum().reset_index()
# time_btwn.to_excel('time_btwn.xlsx')
avg_time=time_btwn.groupby('buyer_username')['time_btwn'].mean().reset_index()

In [21]:
avg_time

,buyer_username,time_btwn
0,adore the couture,1.000000
1,agents in style,3.333333
2,agora vintage,1.500000
3,alexis suitcase,0.000000
4,amanda's exchange,0.000000
...,...,...
246,wgaca,1.100000
247,worth the wait va,1.111111
248,wyld blue,0.000000
249,yolo luxury consignment,1.833333


**seeing if taking out the zero months makes a difference; whenever the first purchase is=automatically 0**

In [22]:
time_btwn2=new[new['time_btwn']!=0].groupby(['buyer_username','month_yr'])['time_btwn'].sum().reset_index()
avg_time2=time_btwn2.groupby('buyer_username')['time_btwn'].mean().reset_index()

avg_time2

,buyer_username,time_btwn
0,adore the couture,1.090909
1,agents in style,5.000000
2,agora vintage,2.000000
3,ambiance,1.333333
4,angie's showroom,6.000000
...,...,...
159,we do vintage,1.000000
160,wgaca,1.222222
161,worth the wait va,1.250000
162,yolo luxury consignment,2.200000


In [23]:
counts2=new.groupby(['buyer_username','month_yr'])['time_btwn'].count().rename('count').reset_index()
updated2=time_btwn2.merge(counts2, how='left')
total_count2=counts2.groupby(['buyer_username'])['count'].sum().reset_index()

full2=avg_time2.merge(total_count2, how='left')

leprix=['carriesherron', 'dianaa', 'karllagerfeld', 'lewisha', 'liveauctioneers_ws', 'lp-mktg', 'lpws-eb', 'rudysromero95', 'shopshops', 'steve', 'tyson',
'rtn+loveluxe', 'loveluxeboutique', 'vivasnobswap', 'mustloveshoes', 'soohee']

full2=full2[~full2['buyer_username'].isin(leprix)]

In [24]:
full2

,buyer_username,time_btwn,count
0,adore the couture,1.090909,56
1,agents in style,5.000000,5
2,agora vintage,2.000000,8
3,ambiance,1.333333,6
4,angie's showroom,6.000000,2
...,...,...,...
159,we do vintage,1.000000,4
160,wgaca,1.222222,52
161,worth the wait va,1.250000,20
162,yolo luxury consignment,2.200000,7


In [25]:
full2.loc[(full2['time_btwn']>=0.5) & (full2['time_btwn']<1.5), 'frequency']='monthly'
full2.loc[(full2['time_btwn']>=1.5) & (full2['time_btwn']<3), 'frequency']='every two months'
full2.loc[(full2['time_btwn']>=3) & (full2['time_btwn']<4), 'frequency']='every three months'
full2.loc[(full2['time_btwn']>=4)]='every four or more months'
# full2.loc[(full2['time_btwn']==0), 'frequency']='bought during one month period'
full2.loc[(full2['count']==1), 'frequency']='one time buyer'

full2['frequency'].value_counts()/full2['frequency'].count()*100

monthly                      55.696203
every two months             31.012658
every four or more months    11.392405
every three months            1.898734
Name: frequency, dtype: float64

-----

**JUST A TEST ABOVE IGNORE**

In [26]:
time_btwn.head()

,buyer_username,month_yr,time_btwn
0,adore the couture,202007,0
1,adore the couture,202008,1
2,adore the couture,202009,1
3,adore the couture,202010,1
4,adore the couture,202011,1


In [27]:
counts=new.groupby(['buyer_username','month_yr'])['total'].sum().rename('count').reset_index()
updated=time_btwn.merge(counts, how='left')
updated

,buyer_username,month_yr,time_btwn,count
0,adore the couture,202007,0,2.0
1,adore the couture,202008,1,37.0
2,adore the couture,202009,1,24.0
3,adore the couture,202010,1,32.0
4,adore the couture,202011,1,11.0
...,...,...,...,...
996,yolo luxury consignment,202104,4,2.0
997,yolo luxury consignment,202105,1,2.0
998,yolo luxury consignment,202106,1,2.0
999,zippychicks,202009,0,2.0


----

## Exact Time Frames

In [151]:
# who purchased EVERY month in the past year?
months=['202007','202008','202009','202010','202011','202012','202101','202102','202103',
        '202104','202105','202106','202107']

og=list(updated[updated['month_yr']=='202007']['buyer_username'].unique())

og_df=updated[updated['buyer_username'].isin(og)]

dropoff={}
start=og.copy()

for m in months: # loop over months 
    sub=og_df[og_df['month_yr']==m] # set subset to current month
    buyers=list(sub['buyer_username'].unique()) # make list of unique buyers from current month
    for b in og: # loop over list of buyers from july 20 
        if b in start:
            if b not in buyers: # check if they are buying in the current month
                start.remove(b) 
                dropoff[b]=m        
print(start)
print(dropoff)

['harley 2019', 'his and her consignment', 'on again consignment', 'one classy shop', 'palm beach princess', 'rue gilt', 'shopbop', 'the digital cantina', 'thrill of the find']
{'byrd designer consignment boutique': '202008', 'city girl consignment': '202008', 'coco et louis': '202008', "hertha's seconds boutique": '202008', 'highbrowbags': '202008', 'hiltons of wilson': '202008', 'lxy boutique': '202008', 'mygirlfriendswardrobe': '202008', 'nh timeless designers': '202008', 'plain jane consignments': '202008', 'reventehcb': '202008', 'store5a': '202008', 'tara tucci': '202008', 'yolo luxury consignment': '202008', 'agora vintage': '202009', 'article consignment': '202009', 'braswell&son': '202009', 'clothes encounters': '202009', 'dcnatickconsignment': '202009', 'dlc': '202009', 'e designer': '202009', "mesee's pieces": '202009', "n'used fashions": '202009', 'onesavvydesign': '202009', 'repeat street': '202009', 'revolve': '202009', 'chic consignment ma': '202010', 'closet exchange ga

In [152]:
freq_counts=og_df.groupby('buyer_username')['month_yr'].count().reset_index()

# percent of total active buyers in past 12 months (dropped of only once or twice)
month_ish=freq_counts[freq_counts['month_yr']>10]
ish_list=list(month_ish['buyer_username'])


len(ish_list)/len(og)*100

32.758620689655174

In [163]:
freq_counts['num_months'] = pd.cut(freq_counts['month_yr'],[0,1,4,7,10,14],
                             labels=['1 month', '2-4 months', '5-7 months',
                                    '8-10 months', '11-13 months'])
freq_counts

,buyer_username,month_yr,num_months
0,adore the couture,12,11-13 months
1,agora vintage,4,2-4 months
2,article consignment,8,8-10 months
3,atelier curated,7,5-7 months
4,bag borrow steal,11,11-13 months
5,braswell&son,8,8-10 months
6,byrd designer consignment boutique,5,5-7 months
7,chic consignment ma,10,8-10 months
8,chic girl,12,11-13 months
9,city girl consignment,4,2-4 months


In [167]:
# freq_counts.groupby('num_months')['buyer_username'].value_counts().to_excel('buyer_freqs_7-12.xlsx')

In [173]:
freq_counts=freq_counts.drop(22)

In [174]:
freq_counts['num_months'].value_counts(normalize=True)*100

11-13 months    33.333333
5-7 months      26.315789
2-4 months      22.807018
8-10 months     12.280702
1 month          5.263158
Name: num_months, dtype: float64

In [153]:
ish_list

['adore the couture',
 'bag borrow steal',
 'chic girl',
 'clothes encounters',
 'e designer',
 'harley 2019',
 'his and her consignment',
 'lucky finds boutique',
 'on again consignment',
 'one classy shop',
 'palm beach princess',
 'real deal collection',
 'rue gilt',
 'shopbop',
 'smitten luxury',
 'tara tucci',
 'the digital cantina',
 'the style lister',
 'thrill of the find']

In [154]:
# dates that buyers dropped off if they didn't buy every single month
ish_off={}
for i in ish_list:
    if i in dropoff:
        ish_off[i]=dropoff[i]

ish_off

{'adore the couture': '202105',
 'bag borrow steal': '202011',
 'chic girl': '202101',
 'clothes encounters': '202009',
 'e designer': '202009',
 'lucky finds boutique': '202103',
 'real deal collection': '202101',
 'smitten luxury': '202107',
 'tara tucci': '202008',
 'the style lister': '202101'}

In [155]:
# percent of total active buyers in past 12 months EXACT out of monthly buyers
len(start)/len(ish_list)*100

47.368421052631575

In [156]:
(og_df[(og_df['buyer_username'].isin(ish_list))].reset_index(drop=True))

,buyer_username,month_yr,time_btwn,count
0,adore the couture,202007,0,2.0
1,adore the couture,202008,1,37.0
2,adore the couture,202009,1,24.0
3,adore the couture,202010,1,32.0
4,adore the couture,202011,1,11.0
...,...,...,...,...
226,thrill of the find,202103,1,13.0
227,thrill of the find,202104,1,12.0
228,thrill of the find,202105,1,1.0
229,thrill of the find,202106,1,9.0


**who is like rue gilt? who is purchasing each month?**

In [157]:
# who purchased EVERY OTHER month in the past year?
months2=['202007','202009','202011','202101','202103',
        '202105','202107']

other=og.copy()

drop_off2={}
for m in months2:
    sub2=og_df[og_df['month_yr']==m]
    buyers2=list(sub2['buyer_username'].unique())
    for b in og:
        if b in other:
            if b not in buyers2:
                other.remove(b)
                drop_off2[b]=m
            elif b in ish_list:
                other.remove(b)
        
print(other)
dropoff

[]


{'byrd designer consignment boutique': '202008',
 'city girl consignment': '202008',
 'coco et louis': '202008',
 "hertha's seconds boutique": '202008',
 'highbrowbags': '202008',
 'hiltons of wilson': '202008',
 'lxy boutique': '202008',
 'mygirlfriendswardrobe': '202008',
 'nh timeless designers': '202008',
 'plain jane consignments': '202008',
 'reventehcb': '202008',
 'store5a': '202008',
 'tara tucci': '202008',
 'yolo luxury consignment': '202008',
 'agora vintage': '202009',
 'article consignment': '202009',
 'braswell&son': '202009',
 'clothes encounters': '202009',
 'dcnatickconsignment': '202009',
 'dlc': '202009',
 'e designer': '202009',
 "mesee's pieces": '202009',
 "n'used fashions": '202009',
 'onesavvydesign': '202009',
 'repeat street': '202009',
 'revolve': '202009',
 'chic consignment ma': '202010',
 'closet exchange ga': '202010',
 'noir blanc boutique llc': '202010',
 "stephanie's luxury consignment": '202010',
 'style encore harrisonburg': '202010',
 'bag borrow s

In [158]:
# percent of total active buyers in past 12 months (dropped of only once or twice)
other_ish=freq_counts[freq_counts['month_yr'].isin([5,6,7,8])]
oth_list=list(other_ish['buyer_username'])

print(len(oth_list)/len(og)*100)

oth_list

34.48275862068966


['article consignment',
 'atelier curated',
 'braswell&son',
 'byrd designer consignment boutique',
 'closet exchange ga',
 'hailey diane luxury resale',
 'loveluxeboutique',
 'marque luxury',
 "n'used fashions",
 'noir blanc boutique llc',
 'pemberley designer consignment',
 'plain jane consignments',
 'repeat street',
 'reventehcb',
 'shoprosejp',
 "stephanie's luxury consignment",
 'style encore harrisonburg',
 'style encore stuart',
 'the lavish loft',
 'yolo luxury consignment']

In [160]:
(og_df[(og_df['buyer_username'].isin(oth_list))].reset_index(drop=True))

**who is purchasing every other month?**

In [36]:
# who purchased EVERY TWO months in the past year?
months3=['202007','202010','202101','202104',
        '202106']

two=og.copy()

for m in months3:
    sub=updated[updated['month_yr']==m]
    buyers=list(sub['buyer_username'].unique())
    for b in og:
        if b in two:
            if b not in buyers:
                two.remove(b)
            if b in start:
                two.remove(b)
            if b in other:
                two.remove(b)
two

['adore the couture',
 'bag borrow steal',
 'chic girl',
 'clothes encounters',
 'e designer',
 'mygirlfriendswardrobe',
 'noir blanc boutique llc',
 'positively radiant boutik',
 'smitten luxury',
 'wgaca']

In [37]:
# who purchased EVERY THREE months in the past year?
months3=['202007','202011','202103','202107']

three=list(updated[updated['month_yr']=='202007']['buyer_username'].unique())
for m in months3:
    sub=updated[updated['month_yr']==m]
    buyers=list(sub['buyer_username'].unique())
    for b in three:
        if b not in buyers:
            three.remove(b)
        if b in start:
            three.remove(b)
        if b in other:
            three.remove(b)
        if b in two:
            three.remove(b)
three

['city girl consignment',
 'highbrowbags',
 'lxy boutique',
 'pemberley designer consignment',
 'real deal collection',
 'store5a']

In [38]:
len(updated['buyer_username'].unique())

251

----

In [39]:
total_count=counts.groupby(['buyer_username'])['count'].sum().reset_index()

full=avg_time.merge(total_count, how='left')
full

,buyer_username,time_btwn,count
0,adore the couture,1.000000,142.0
1,agents in style,3.333333,13.0
2,agora vintage,1.500000,31.0
3,alexis suitcase,0.000000,14.0
4,amanda's exchange,0.000000,16.0
...,...,...,...
246,wgaca,1.100000,478.0
247,worth the wait va,1.111111,65.0
248,wyld blue,0.000000,195.0
249,yolo luxury consignment,1.833333,13.0


In [40]:
full.shape

(251, 3)

In [41]:
leprix=['carriesherron', 'dianaa', 'karllagerfeld', 'lewisha', 'liveauctioneers_ws', 'lp-mktg', 'lpws-eb', 'rudysromero95', 'shopshops', 'steve', 'tyson',
'rtn+loveluxe', 'loveluxeboutique', 'vivasnobswap', 'mustloveshoes', 'soohee']

full=full[~full['buyer_username'].isin(leprix)]

print(leprix)
full.shape

['carriesherron', 'dianaa', 'karllagerfeld', 'lewisha', 'liveauctioneers_ws', 'lp-mktg', 'lpws-eb', 'rudysromero95', 'shopshops', 'steve', 'tyson', 'rtn+loveluxe', 'loveluxeboutique', 'vivasnobswap', 'mustloveshoes', 'soohee']


(241, 3)

Dataset includes buyer username, average number of months between purchases, and total number of purchases

In [42]:
full.loc[(full['time_btwn']>=0.5) & (full['time_btwn']<1.5), 'frequency']='monthly'
full.loc[(full['time_btwn']>=1.5) & (full['time_btwn']<3), 'frequency']='every two months'
full.loc[(full['time_btwn']>=3) & (full['time_btwn']<4), 'frequency']='every three months'
full.loc[(full['time_btwn']>=4),'frequency']='every four or more months'
full.loc[(full['time_btwn']==0), 'frequency']='bought during one month period'
full.loc[(full['count']==1), 'frequency']='one time buyer'

full.to_excel('full_frequencies.xlsx')

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [43]:
full['frequency'].value_counts()/full['frequency'].count()*100

monthly                           46.473029
bought during one month period    23.651452
every two months                  14.107884
one time buyer                    10.788382
every three months                 4.149378
every four or more months          0.829876
Name: frequency, dtype: float64

In [44]:
# q2=pd.read_csv('q2b2c.csv')